In [37]:
# 라이브러리 import
! pip install bs4
! pip install pandas
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import pprint

In [38]:

### 중앙부처 데이터만 저장할 수 있도록 만든 코드입니다. 지자체나 지역별 데이터 저장을 원하면 수정하여 활용하시길 바랍니다.
## 1. 온통 청년 API로 정책 html 불러오기
key = 'YOUR_API_KEY' ### 여기에 신청한 본인의 key를 넣어야 합니다!
page = ['0', '1', '2', '3', '4', '5'] # 조회할 페이지, 총 정책 수가 업데이트로 인해 늘어나면 현재 '0' ~ '4'인 페이지를 추가해야 할 수도 있음. 예) page = ['0', '1', '2', '3', '4', '5']
outputNum = '100' # 출력건수, 기본 10 최대 100
#region = '003002015' # 지역별로 조회하고 싶다면 지역 코드를 지정하면 됨. 여러 지역을 동시에 조회하고 싶다면 쉼표(,)로 코드를 연결해 주면 됨. 예) '003002015,003002014'

contents = ''
for p in page:
    #queryParams = 'pageIndex=' + page + '&display=' + outputNum + '&openApiVlak=' + key + '&srchPolyBizSecd=' + region # 지역별로 조회하게 되면 page를 더 적게 지정해야 할 수도 있음.
    queryParams = 'pageIndex=' + p + '&display=' + outputNum + '&openApiVlak=' + key
    url = 'https://www.youthcenter.go.kr/opi/youthPlcyList.do?'+ queryParams

    # url 불러오기
    response = requests.get(url)
    #print(type(response))
    print(len(response.text)) # 130으로 나오는 경우 해당 페이지에 출력할 정책이 없는 경우임, 즉 130 이전 page까지만 포함시키면 됨.

    #데이터 값 출력해보기
    contents += response.text
    #print(type(contents)) # string 객체로 바뀜

262038
262038
249243
250392
257486
154743


In [39]:
# 데이터 결과값 예쁘게 출력해주는 코드
pp = pprint.PrettyPrinter(indent=4, depth=None)
print(pp.pprint(contents[:3000]))

('<?xml version="1.0" encoding="UTF-8"?>\n'
 '<youthPolicyList>\n'
 '   <pageIndex>1</pageIndex>\n'
 '   <totalCnt>459</totalCnt>\n'
 '   <youthPolicy>\n'
 '      <rnum>1</rnum>\n'
 '      <bizId><![CDATA[R2024030420277]]></bizId>\n'
 '      <polyBizSecd>003001016002</polyBizSecd>\n'
 '      <polyBizTy>중앙부처</polyBizTy>\n'
 '      <polyBizSjnm><![CDATA[산모·신생아 건강관리 지원사업]]></polyBizSjnm>\n'
 '      <polyItcnCn><![CDATA[출산가정에 건강관리사를 파견하여 산후관리를 지원함으로써 산모와 신생아의 건강을 증진하고 '
 '출산가정의 경제적 부담을 경감합니다.]]></polyItcnCn>\n'
 '      <sporCn><![CDATA[□ 정부 바우처 이용자에게 기본적으로 제공되는 표준화된 서비스 지원\r\n'
 '※ 자세한 내용은 첨부된 파일을 참고]]></sporCn>\n'
 '      <sporScvl><![CDATA[-]]></sporScvl>\n'
 '      <bizPrdCn><![CDATA[2024.01.01 ~ 2024.12.31]]></bizPrdCn>\n'
 '      <prdRpttSecd><![CDATA[002001]]></prdRpttSecd>\n'
 '      <rqutPrdCn><![CDATA[-]]></rqutPrdCn>\n'
 '      <ageInfo><![CDATA[제한없음]]></ageInfo>\n'
 '      <majrRqisCn><![CDATA[-]]></majrRqisCn>\n'
 '      <empmSttsCn><![CDATA[-]]></empmSttsCn>\n'
 '      <splzRl

In [40]:
## 2. BeautifulSoup으로 html parsing
soup = BeautifulSoup(contents, 'html.parser')
#finded_values=soup.find_all('ageInfo')
#[x.text for x in finded_values] # .text를 통해 안의 내용물만을 불러옵니다

/home/gyuni/miniforge3/envs/final_env/lib/python3.9/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [41]:
print('현재 총 정책 수:', soup.find('totalcnt').text.replace('<totalcnt>', '').replace('</totalcnt>', ''))

현재 총 정책 수: 459


In [42]:
## 3. df 생성
YouthPolicyInfo = {}
# attr 이름 모두 소문자로
#attr_to_find_list=['polybizsjnm','sporcn', 'ageinfo','majrrqiscn','empmsttscn','splzrlmrqiscn','accrrqiscn','prcpcn','aditrscn','prcplmtttrgtcn']
attr_to_find_list=['rnum', 'bizid', 'polybizsecd', 'polybizty', 'polybizsjnm', 'polyitcncn', 'sporcn', 'sporscvl', 'bizprdcn', 'prdrpttsecd', 'rqutprdcn', 
                   'ageinfo','majrrqiscn','empmsttscn','splzrlmrqiscn','accrrqiscn','prcpcn','aditrscn','prcplmtttrgtcn',
                   'rqutproccn', 'pstnpaprcn', 'jdgnprescn', 'rquturla', 'rfcsiteurla1', 'rfcsiteurla2', 'mngtmson', 'mngtmrofchercn', 'cherctpccn',
                   'cnsgnmor', 'tintchercn', 'tintcherctpccn', 'etct', 'polyrlmcd'] # 모두 소문자여야 함.
print(len(attr_to_find_list))

for each_attr in attr_to_find_list:
    finded_attr=soup.find_all(each_attr)
    YouthPolicyInfo[each_attr]=[x.text for x in finded_attr]

df = pd.DataFrame(YouthPolicyInfo)
#alt_colnames = ['정책명', '지원내용', '연령', '전공요건', '취업상태', '특화분화', '학력', '거주지및소득', '추가단서', '참여제한']
alt_colnames = ['번호', '정책 ID', '정책 일련번호', '기관 및 지자체 구분', '정책명', '정책 소개', '지원 내용', '지원 규모', '사업 운영 기간', '사업 신청 기간 반복 구분 코드', '사업 신청 기간', 
                '연령', '전공', '취업 상태', '특화 분화', '학력', '거주지 및 소득', '추가 단서 사항', '참여 제한 대상', 
                '신청 절차', '제출 서류', '심사 발표', '신청 사이트 주소', '참고 사이트 주소1', '참고 사이트 주소2', '주관 부처명', '주관 부처 담당자 이름', '주관 부처 담당자 연락처',
                '운영 기관명', '운영 기관 담당자 이름', '운영 기관 담당자 연락처', '기타 사항', '정책 분야 코드']
print(len(alt_colnames))

df.columns = alt_colnames

33
33


In [43]:
print(df.shape)
df.head()

(559, 33)


,번호,정책 ID,정책 일련번호,기관 및 지자체 구분,정책명,정책 소개,지원 내용,지원 규모,사업 운영 기간,사업 신청 기간 반복 구분 코드,...,참고 사이트 주소1,참고 사이트 주소2,주관 부처명,주관 부처 담당자 이름,주관 부처 담당자 연락처,운영 기관명,운영 기관 담당자 이름,운영 기관 담당자 연락처,기타 사항,정책 분야 코드
0,1,R2024030420277,003001016002,중앙부처,산모·신생아 건강관리 지원사업,출산가정에 건강관리사를 파견하여 산후관리를 지원함으로써 산모와 신생아의 건강을 증진...,□ 정부 바우처 이용자에게 기본적으로 제공되는 표준화된 서비스 지원\r\n※ 자세한...,-,2024.01.01 ~ 2024.12.31,002001,...,-,-,사회서비스사업과,김현아,044-202-3223,사회서비스사업과,김현아,044-202-3223,-,023040
1,2,R2024030420290,003002014019,지자체,전세보증금반환보증 보증료 지원,전세보증금반환보증 보증료 지원(최대 30만원),□ 지원 내용\r\n○ 전세보증금반환보증 보증료 지원(최대 30만원)\r\n- 청년...,null,2024-03-04~2024-12-31,002004,...,http://gbyouth.co.kr/policy/list.tc?mn=2379&pa...,-,국토교통부,-,칠곡군 일자리경제과 (054-979-6547),칠곡군,-,칠곡군 일자리경제과 (054-979-6547),-,023020
2,3,R2024030420289,003002014018,지자체,전세보증금반환보증 보증료 지원,전세보증금반환보증 보증료 지원(최대 30만원),□ 지원 내용\r\n○ 전세보증금반환보증 보증료 지원(최대 30만원)\r\n- 청년...,null,2024-03-04~2024-12-31,002004,...,http://gbyouth.co.kr/policy/list.tc?mn=2379&pa...,-,국토교통부,-,성주군 일자리지원팀(054-930-6712),성주군,-,성주군 일자리지원팀(054-930-6712),-,023020
3,4,R2024030420276,003002014013,지자체,전세보증금반환보증 보험료 지원,전세보증금반환보증 보증료 지원(최대 30만원),□ 지원 내용\r\n○ 전세보증금반환보증 보증료 지원(최대 30만원)\r\n- 청년...,null,2024-03-04~2024-12-31,002004,...,http://gbyouth.co.kr/policy/list.tc?mn=2379&pa...,-,국토교통부,-,청송군청 기획감사실 행복일자리팀(054-870-6454),청송군,-,청송군청 기획감사실 행복일자리팀(054-870-6454),-,023020
4,5,R2024030420275,003002014005,지자체,전세보증금반환보증 보증료 지원,전세보증금반환보증 보증료 지원(최대 30만원),□ 지원 내용\r\n○ 전세보증금반환보증 보증료 지원(최대 30만원)\r\n- 청년...,null,2024-03-04~2024-12-31,002004,...,http://gbyouth.co.kr/policy/list.tc?mn=2379&pa...,-,국토교통부,-,구미시 인구청년과(054-480-2523),구미시,-,구미시 인구청년과(054-480-2523),-,023020


In [44]:
## 4. 부족한 정보 추가
# 4-1. 정책 분야 추가
polysec = []
for code in df['정책 분야 코드']:
    if code == '023010':
        polysec.append('일자리 분야')
    elif code == '023020':
        polysec.append('주거 분야')
    elif code == '023030':
        polysec.append('교육 분야')
    elif code == '023040':
        polysec.append('복지·문화 분야')
    elif code == '023050':
        polysec.append('참여·권리 분야')
    else:
        polysec.append('코드 매칭 분야 없음')

df['정책 분야'] = polysec

In [45]:
# 4-2. 온통 청년 url 추가
df['온통 청년 url'] = "https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?pageIndex=1&frameYn=&bizId=" + df['정책 ID'] + "&dtlOpenYn=&plcyTpOpenTy=&plcyCmprInfo=&srchWord=&chargerOrgCdAll=&srchAge=&trgtJynEmp=&trgtJynEmp=&srchSortOrder=1&pageUnit=12"

In [46]:
print(df.shape)
df.head()

(559, 35)


,번호,정책 ID,정책 일련번호,기관 및 지자체 구분,정책명,정책 소개,지원 내용,지원 규모,사업 운영 기간,사업 신청 기간 반복 구분 코드,...,주관 부처명,주관 부처 담당자 이름,주관 부처 담당자 연락처,운영 기관명,운영 기관 담당자 이름,운영 기관 담당자 연락처,기타 사항,정책 분야 코드,정책 분야,온통 청년 url
0,1,R2024030420277,003001016002,중앙부처,산모·신생아 건강관리 지원사업,출산가정에 건강관리사를 파견하여 산후관리를 지원함으로써 산모와 신생아의 건강을 증진...,□ 정부 바우처 이용자에게 기본적으로 제공되는 표준화된 서비스 지원\r\n※ 자세한...,-,2024.01.01 ~ 2024.12.31,002001,...,사회서비스사업과,김현아,044-202-3223,사회서비스사업과,김현아,044-202-3223,-,023040,복지·문화 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
1,2,R2024030420290,003002014019,지자체,전세보증금반환보증 보증료 지원,전세보증금반환보증 보증료 지원(최대 30만원),□ 지원 내용\r\n○ 전세보증금반환보증 보증료 지원(최대 30만원)\r\n- 청년...,null,2024-03-04~2024-12-31,002004,...,국토교통부,-,칠곡군 일자리경제과 (054-979-6547),칠곡군,-,칠곡군 일자리경제과 (054-979-6547),-,023020,주거 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
2,3,R2024030420289,003002014018,지자체,전세보증금반환보증 보증료 지원,전세보증금반환보증 보증료 지원(최대 30만원),□ 지원 내용\r\n○ 전세보증금반환보증 보증료 지원(최대 30만원)\r\n- 청년...,null,2024-03-04~2024-12-31,002004,...,국토교통부,-,성주군 일자리지원팀(054-930-6712),성주군,-,성주군 일자리지원팀(054-930-6712),-,023020,주거 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
3,4,R2024030420276,003002014013,지자체,전세보증금반환보증 보험료 지원,전세보증금반환보증 보증료 지원(최대 30만원),□ 지원 내용\r\n○ 전세보증금반환보증 보증료 지원(최대 30만원)\r\n- 청년...,null,2024-03-04~2024-12-31,002004,...,국토교통부,-,청송군청 기획감사실 행복일자리팀(054-870-6454),청송군,-,청송군청 기획감사실 행복일자리팀(054-870-6454),-,023020,주거 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...
4,5,R2024030420275,003002014005,지자체,전세보증금반환보증 보증료 지원,전세보증금반환보증 보증료 지원(최대 30만원),□ 지원 내용\r\n○ 전세보증금반환보증 보증료 지원(최대 30만원)\r\n- 청년...,null,2024-03-04~2024-12-31,002004,...,국토교통부,-,구미시 인구청년과(054-480-2523),구미시,-,구미시 인구청년과(054-480-2523),-,023020,주거 분야,https://www.youthcenter.go.kr/youngPlcyUnif/yo...


In [47]:
# 정책 분야 카테고리 확인
df['정책 분야'].unique()

array(['복지·문화 분야', '주거 분야', '교육 분야', '일자리 분야', '참여·권리 분야'], dtype=object)

In [48]:
## 5. 중복 데이터 제거
# 5-1. 중복 데이터 확인
print(df['정책명'].value_counts())
#df['정책명'].unique()

정책명
전세보증금반환보증 보증료 지원                        34
2024년 상반기 예술창작공간 입주작가 모집                 8
청년도전지원사업                                 3
2024년 청년도전 지원사업                          3
청춘사진관 이력서사진 무료 촬영(양주시)                   2
                                        ..
(대학생) 2024년 경상남도청소년참여위원회 모집              1
2024년 생애최초 청년창업 지원사업 예비창업자 모집 공고(수정)     1
2024 충남관광 스타트업 육성지원                      1
성남시 청년 재창업 지원                            1
학자금대출 신용유의자 신용회복 지원사업                    1
Name: count, Length: 433, dtype: int64


In [49]:
# 5-2. 중복 제거
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(559, 35)
(459, 35)


In [50]:
## 6. 중앙부처 데이터만 가져오기
# 6-1. 중앙부처/지자체 구분 확인
df['기관 및 지자체 구분'].unique()

array(['중앙부처', '지자체'], dtype=object)

In [51]:
# 6-2. 중앙부처 정책만 있는 df 생성
df_head = df[df['기관 및 지자체 구분']=='중앙부처']

In [52]:
from datetime import datetime

# 현재 날짜와 시간을 얻기
current_date = datetime.now()

# 현재 날짜를 YYYY-MM-DD 형식으로 출력
formatted_date = current_date.strftime('%Y-%m-%d')
formatted_date

'2024-03-04'

In [53]:
import os

# 현재 작업 중인 폴더의 경로를 얻기
current_path = os.getcwd()
current_path

'/home/gyuni/projects/level2-3-nlp-finalproject-nlp-03/Jupyter'

In [54]:
# 7. csv 저장하기 
df_head.to_csv(f"{current_path}/중앙부처_{formatted_date}.csv")